5- Hypothetical model for the turbine data. 

Directory is changed to get all the batches

In [1]:
# Adding all needed imports

import tensorflow as tf
from tensorflow import keras
from keras import regularizers
from keras import layers
from keras.models import Model
#print(tf.keras.__version__)
#print(tf.__version__)
import numpy as np
import pandas as pd
import glob,os
import pickle
import random as rng


dir = "D:\\WindData\\POD_Request_3541\\Project\\processed\\trimmed\\final"

os.chdir(dir)

Using TensorFlow backend.


Get the list of batches as well as loading the list of labels

In [2]:
files = glob.glob("./data/*.npy")

In [3]:
avgLabels = pickle.load( open('./labels.p', "rb" ) )

Specify the batch generator. The model calls the generator each batch for the data and labels. The data should be autoshuffled every time a new epoch starts.

In [4]:
#Shuffle the list of files so that the batches are in a different order each epoch
def shuffle(trainFiles, avgLabels):
    newFileList = []
    newLabelList = []
    for i in range(len(trainFiles)):
        index = rng.randint(0, len(trainFiles)-1)
        newFileList.append(trainFiles.pop(index))
        newLabelList.append(avgLabels.pop(index))
    trainFiles = newFileList
    avgLabels = newLabelList
    return trainFiles, avgLabels

#Batch generator. Data and labels are retrieved from the data file and label list respectively. Yield call should make it loop infinitely as the model requires
def generator(trainFiles, avgLabels):
    num_samples = len(trainFiles)
    
    while True:
        trainFiles, avgLabels = shuffle(trainFiles, avgLabels)
        for i in range(num_samples):
            batchTrain = np.load(trainFiles[i])
            batchLabels = np.asarray(avgLabels[i])
            yield batchTrain, batchLabels
            
train_generator = generator(files, avgLabels) 

Model creation, compiling and calling the fit function. Sequential isn't used to make it possible to experiment with parallel neural nets

In [5]:
width = 31
numRows = 60

In [6]:
visible = layers.Input(shape=(numRows, width,))

cnn = layers.Conv1D(150, 6, activation='relu')(visible)
rnn1 = layers.GRU(300, dropout=0.2)(cnn)



dense = layers.Dense(150, activation='relu')(rnn1)
drop = layers.Dropout(0.2)(dense)
out = layers.Dense(1, activation='linear')(drop)

model = Model(inputs=visible, outputs=out)


print(model.summary())


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 60, 31)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 55, 150)           28050     
_________________________________________________________________
gru_1 (GRU)                  (None, 300)               405900    
_________________________________________________________________
dense_1 (Dense)              (None, 150)               45150     
_________________________________________________________________
dropout_1 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 151       
Total params: 479,251
Trainable params: 479,251
Non-trainable params: 0
_____________________________________________________

In [7]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9),
              loss='mae', 
              metrics=['mae'])

In [8]:
history = model.fit_generator(train_generator, epochs=5, verbose=1, steps_per_epoch=len(files))

Epoch 1/5
1700/1700 [==============================] - 496s 292ms/step - loss: 232268.6635 - mae: 232268.7500
Epoch 2/5
1700/1700 [==============================] - 480s 283ms/step - loss: 206600.8080 - mae: 206600.8125
Epoch 3/5
1700/1700 [==============================] - 430s 253ms/step - loss: 186168.4979 - mae: 186168.6562
Epoch 4/5
1700/1700 [==============================] - 418s 246ms/step - loss: 171760.5677 - mae: 171760.2969
Epoch 5/5
1700/1700 [==============================] - 420s 247ms/step - loss: 164780.8756 - mae: 164780.8906
